# Libraries and functions

In [ ]:
#import required libraries
import numpy as np
import pandas as pd
import folium
import geopandas as gpd
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import FloatImage
from shapely.geometry import box
from shapely.geometry import MultiLineString
import branca
from colour import Color
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
#CODE BORROWED FROM https://nbviewer.jupyter.org/gist/BibMartin/f153aa957ddc5fadc64929abdee9ff2e
from branca.element import MacroElement

from jinja2 import Template

class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)  # noqa

In [ ]:
#function for creating heatmap given lat, long and weight data
def create_heatmap(list_lat, list_long, map_osm, heat_name, list_weight = [], r = 20):
    #create feature layer
    feature = folium.map.FeatureGroup(name = heat_name, overlay = True, show = False)

    #if weights are given
    if (len(list_weight) > 0):
        #create heatmap
        plugins.HeatMap(zip(list_lat, list_long, list_weight), radius = r, gradient = {0.25: "blue", 0.4: "lightblue", 0.6: "lime", 0.8: "yellow", 1.0: "red"}).add_to(feature)

        #create legend
        legend = create_legend(list_weight)

        #add feature to map and bind feature and legend together
        map_osm.add_child(feature)
        map_osm.add_child(legend)
        map_osm.add_child(BindColormap(feature,legend))
    
    #otherwise don't use any weights
    else:
        #create heatmap
        plugins.HeatMap(zip(list_lat, list_long), radius = r, gradient = {0.25: "blue", 0.4: "lightblue", 0.6: "lime", 0.8: "yellow", 1.0: "red"}).add_to(feature)

        #create legend
        legend = create_legend()

        #add feature to map and bind feature and legend together
        map_osm.add_child(feature)
        map_osm.add_child(legend)
        map_osm.add_child(BindColormap(feature,legend))


In [ ]:
#function for creating heatmap given lat, long and weight data (specifically for incidents)
def create_heatmap_incidents(list_lat, list_long, map_osm, heat_name, list_weight = [], r = 20):
    #create feature layer
    feature = folium.map.FeatureGroup(name = heat_name, overlay = True, show = False)

    #if weights are given
    if (len(list_weight) > 0):
        #create heatmap
        
        #inferno color scheme
        # plugins.HeatMap(zip(list_lat, list_long), radius = r, gradient = {0.025: "#2f0087", 0.0714285715: "#6200a4", 0.142857143: "#9200a6", 0.214285: "#ba2f8a", 0.28571: "#d85b69", 0.35714: "#ee8949", 0.42855: "#f6bd27", 0.5: "#e4fa15"}).add_to(feature)
        # plugins.HeatMap(zip(list_lat, list_long), radius = r, gradient = {0.05: "#2f0087", 0.142857143: "#6200a4", 0.285714286: "#9200a6", 0.428571429: "#ba2f8a", 0.571428571: "#d85b69", 0.714285714: "#ee8949", 0.857142857: "#f6bd27", 1.0: "#e4fa15"}).add_to(feature)

        #inferno reversed
        # plugins.HeatMap(zip(list_lat, list_long), radius = r, gradient = {0.025: "#e4fa15", 0.0714285715: "#f6bd27", 0.142857143: "#ee8949", 0.214285: "#d85b69", 0.28571: "#ba2f8a", 0.35714: "#9200a6", 0.42855: "#6200a4", 0.5: "#2f0087"}).add_to(feature)
        plugins.HeatMap(zip(list_lat, list_long), radius = r, gradient = {0.05: "#e4fa15", 0.142857143: "#f6bd27", 0.285714286: "#ee8949",  0.428571429: "#d85b69", 0.571428571: "#ba2f8a", 0.714285714: "#9200a6", 0.857142857: "#6200a4", 1.0: "#2f0087"}).add_to(feature)

        #create legend
        legend = create_legend_incidents(list_weight)

        #add feature to map and bind feature and legend together
        map_osm.add_child(feature)
        map_osm.add_child(legend)
        map_osm.add_child(BindColormap(feature,legend))
    
    #otherwise don't use any weights
    else:
        #create heatmap

        #inferno color scheme
        # plugins.HeatMap(zip(list_lat, list_long), radius = r, gradient = {0.025: "#2f0087", 0.0714285715: "#6200a4", 0.142857143: "#9200a6", 0.214285: "#ba2f8a", 0.28571: "#d85b69", 0.35714: "#ee8949", 0.42855: "#f6bd27", 0.5: "#e4fa15"}).add_to(feature)
        # plugins.HeatMap(zip(list_lat, list_long), radius = r, gradient = {0.05: "#2f0087", 0.142857143: "#6200a4", 0.285714286: "#9200a6", 0.428571429: "#ba2f8a", 0.571428571: "#d85b69", 0.714285714: "#ee8949", 0.857142857: "#f6bd27", 1.0: "#e4fa15"}).add_to(feature)

        #inferno reversed
        # plugins.HeatMap(zip(list_lat, list_long), radius = r, gradient = {0.025: "#e4fa15", 0.0714285715: "#f6bd27", 0.142857143: "#ee8949", 0.214285: "#d85b69", 0.28571: "#ba2f8a", 0.35714: "#9200a6", 0.42855: "#6200a4", 0.5: "#2f0087"}).add_to(feature)
        plugins.HeatMap(zip(list_lat, list_long), radius = r, gradient = {0.05: "#e4fa15", 0.142857143: "#f6bd27", 0.285714286: "#ee8949",  0.428571429: "#d85b69", 0.571428571: "#ba2f8a", 0.714285714: "#9200a6", 0.857142857: "#6200a4", 1.0: "#2f0087"}).add_to(feature)

        #create legend
        legend = create_legend_incidents()

        #add feature to map and bind feature and legend together
        map_osm.add_child(feature)
        map_osm.add_child(legend)
        map_osm.add_child(BindColormap(feature,legend))

In [ ]:
#function for generating legend (returns a linear interpolated color gradient)
def create_legend(list_weight = []):
    #colors and their respective index values
    colors = ["blue", "lightblue", "lime", "yellow", "red"]
    index = [0.25, 0.4, 0.6, 0.8, 1.0]
    
    #if weights are given, scale the index
    if (len(list_weight) > 0):
        #scale index with weight values
        max_weight = max(list_weight)
        min_weight = min(list_weight)
        diff_weight = max_weight - min_weight
        index = [(i*diff_weight) + min_weight for i in index]

        #create a linear interpolation of colors
        colormap = branca.colormap.LinearColormap(colors = colors, index = index, vmin = min_weight, vmax = max_weight)
    
    #else, use standard index
    else:
        colormap = branca.colormap.LinearColormap(colors = colors, index = index)

    return colormap

In [ ]:
#function for generating legend (returns a linear interpolated color gradient) (specifically for incidents)
def create_legend_incidents(list_weight = []):
    #colors and their respective index values
    # colors = ["#2f0087", "#6200a4", "#9200a6", "#ba2f8a", "#d85b69", "#ee8949", "#f6bd27", "#e4fa15"]
    colors = ["#e4fa15", "#f6bd27", "#ee8949", "#d85b69", "#ba2f8a", "#9200a6", "#6200a4", "#2f0087"]
    index = [0.05, 0.142857143, 0.285714286, 0.428571429, 0.571428571, 0.714285714, 0.857142857, 1.0]
    
    #if weights are given, scale the index
    if (len(list_weight) > 0):
        #scale index with weight values
        max_weight = max(list_weight)
        min_weight = min(list_weight)
        diff_weight = max_weight - min_weight
        index = [(i*diff_weight) + min_weight for i in index]

        #create a linear interpolation of colors
        colormap = branca.colormap.LinearColormap(colors = colors, index = index, vmin = min_weight, vmax = max_weight)
    
    #else, use standard index
    else:
        colormap = branca.colormap.LinearColormap(colors = colors, index = index)

    return colormap

In [ ]:
#function for creating grid markers
def create_markers(map_osm, grid_arr):
    #create feature layer
    feature = folium.map.FeatureGroup(name = "grid markers", overlay = True, show = True)

    #open combined csv file and replace nan with string NaN
    combined_df = pd.read_csv("..\..\CSV_files\Combined_Data.csv")
    combined_df.fillna("NaN", inplace = True)

    #iterate through rows
    counter = 0
    for index, row in combined_df.iterrows():
        #find middle point of grid
        # grid = grid_arr[row["Grid"]]
        long_coord = (grid_arr[counter][0][0] + grid_arr[counter][3][0]) / 2
        lat_coord = (grid_arr[counter][0][1] + grid_arr[counter][3][1]) / 2
        counter += 1
        column_names = list(combined_df.columns.values)

        #create string of row data
        text = ""
        for col_name in column_names:
            text += "<br>" + col_name + ": " + str(row[col_name]) + "<br>"

        #add to feature layer
        iframe = folium.IFrame(text, width=220, height=310)
        popup = folium.Popup(iframe)
        folium.Marker(location=[lat_coord, long_coord], popup=popup).add_to(feature)
 
    
    #add to map
    map_osm.add_child(feature)

In [ ]:
#function for adding Calgary 10x10 grid array onto the map
def create_grid_array(grid_array, map_osm):
    #create feature layer
    feature = folium.map.FeatureGroup(name = "grid array", overlay = True, show = True)

    #iterate through each grid
    for grid in grid_array:
        #find the 4 corners of the grid
        coord_box = box(grid[0][0],grid[0][1],grid[3][0],grid[3][1])

        #create box geojson and add to feature layer
        geo = gpd.GeoSeries([coord_box]).__geo_interface__
        folium.GeoJson(geo).add_to(folium.FeatureGroup(name='grid array')).add_to(feature)
    
    #add feature layer to map
    map_osm.add_child(feature)

# Creating 10x10 grid

In [ ]:
#Read city boundary layer
df = pd.read_csv("..\..\CSV_files\City_Boundary_layer.csv")
df

In [ ]:
#parse the coordinates
string = df["the_geom"].values[0]
string_stripped= string.replace("POLYGON","").replace("(","").replace(")","").replace(",","")
string_split = string_stripped.split()

In [ ]:
list_long = []
list_lat = []

#appends latitudes and longitudes based on long/lat/long/lat... pattern
for i in range(len(string_split)):
    if i % 2 == 0:
        list_long.append(float(string_split[i]))
    else:
        list_lat.append(float(string_split[i]))

#find the min and max values of the boundary
min_long = min(list_long)
max_long = max(list_long)
min_lat = min(list_lat)
max_lat = max(list_lat)
print(min_long)
print(max_long)

In [ ]:
#creation of grid array for a 10x10 grid
long_ten_split = np.linspace(min_long,max_long,num = 11)
lat_ten_split = np.linspace(min_lat,max_lat,num = 11)

grid_array = []
for y in range(len(lat_ten_split)-1):
    for x in range(len(long_ten_split)-1):
        bot_left = [long_ten_split[x],lat_ten_split[y]]
        bot_right = [long_ten_split[x+1],lat_ten_split[y]]
        top_left = [long_ten_split[x],lat_ten_split[y+1]]
        top_right = [long_ten_split[x+1],lat_ten_split[y+1]]
        grid_array.append([bot_left,bot_right,top_left,top_right])

# Traffic volume heat map

In [ ]:
#read traffic csv
traffic_df = pd.read_csv("..\..\CSV_files\Traffic_Volumes_for_2018.csv")
traffic_df

In [ ]:
#parse data for lat, longs and weight (volume)
traffic_data = []

#parse multilinestring
for string,volume in zip(traffic_df['multilinestring'],traffic_df["VOLUME"]):
    string_stripped= string.replace("MULTILINESTRING","").replace("(","").replace(")","").replace(",","")
    string_split = string_stripped.split()
    float_split = [float(i) for i in string_split]

    for i in range(int(len(float_split)/2)):
        traffic_data.append([float_split[i*2+1],float_split[i*2],float(volume)])


list_lat = [i[0] for i in traffic_data]
list_long = [i[1] for i in traffic_data]
list_weight = [i[2] for i in traffic_data]

In [ ]:
#create map
map_osm = folium.Map(location = [50.913577283979,-114.073657541927],control_scale = True, zoom_start=10)

#add heatmap
create_heatmap(list_lat, list_long, map_osm, heat_name = "volume", list_weight = list_weight)

# create_grid_array(grid_array, map_osm)
# create_markers(map_osm, grid_array)
# folium.LayerControl().add_to(map_osm)

# map_osm.save("traffic_heat_map_grid_marker.html")
# map_osm

# Traffic camera heat map

In [ ]:
#read traffic camera locations csv
camera_df = pd.read_csv("..\..\CSV_files\Traffic_Camera_Locations.csv")
camera_df

In [ ]:
#parse data for lats and longs
list_long = [i for i in camera_df["longitude"]]
list_lat = [i for i in camera_df["latitude"]]

In [ ]:
# map_osm = folium.Map(location = [50.913577283979,-114.073657541927],control_scale = True, zoom_start=10)

#add heatmap
create_heatmap(list_lat, list_long, map_osm, heat_name = "camera", r = 25)
# create_grid_array(grid_array, map_osm)
# create_markers(map_osm, grid_array)
# folium.LayerControl().add_to(map_osm)

# map_osm.save("traffic_heat_map_grid_marker.html")
# map_osm

# Traffic signals heat map

In [ ]:
#read traffic signals csv
signals_df = pd.read_csv("..\..\CSV_files\Traffic_Signals.csv")
signals_df

In [ ]:
#parse data for lats and longs
list_long = [i for i in signals_df["longitude"]]
list_lat = [i for i in signals_df["latitude"]]

In [ ]:
#create map
# map_osm = folium.Map(location = [50.913577283979,-114.073657541927],control_scale = True, zoom_start=10)

#add heatmap
create_heatmap(list_lat, list_long, map_osm, heat_name = "signals", r = 17)
# create_grid_array(grid_array, map_osm)
# create_markers(map_osm, grid_array)
# folium.LayerControl().add_to(map_osm)

# map_osm.save("traffic_heat_map_grid_marker.html")
# map_osm

# Traffic signs heat map

In [ ]:
#read traffic signs csv
signs_df = pd.read_csv("..\..\CSV_files\Traffic_Signs.csv")

#drop nan and 0 values in sign count and sign type
signs_df.dropna(subset=["SGN_COUNT_NO"], how='all', inplace=True)
signs_df.dropna(subset=["BLADE_TYPE"], how='all', inplace=True)
signs_df = signs_df[signs_df.SGN_COUNT_NO != 0]

#list of "irrelevant signs" (parking signs, info signs etc.)
signs_nan = ['Timed Parking','Park Plus','Parking Restrictions','Street Name','Snow Route','Guide / Information','Loading Zone','Residential Parking', 'Overhead Guide']

#remove irrelevant signs
signs_df = signs_df.loc[~signs_df["BLADE_TYPE"].isin(signs_nan)]

signs_df

In [ ]:
#parse data for lats, longs and weight (sign count)
list_long = []
list_lat = []
list_weight = [i for i in signs_df["SGN_COUNT_NO"]]

#parse POINT string
for row in signs_df["POINT"]:
    string_stripped = row.replace("POINT", "").replace("(", "").replace(")", "")
    string_split = string_stripped.split()

    list_long.append(float(string_split[0]))
    list_lat.append(float(string_split[1]))

In [ ]:
#create map
# map_osm = folium.Map(location = [50.913577283979,-114.073657541927],control_scale = True, zoom_start=10)

#add heatmap
create_heatmap(list_lat, list_long, map_osm, heat_name = "signs (all)", list_weight = list_weight, r = 15)
# create_grid_array(grid_array, map_osm)
# create_markers(map_osm, grid_array)
# folium.LayerControl().add_to(map_osm)

# map_osm.save("traffic_heat_map_grid_marker.html")
# map_osm

# Traffic incidents

In [ ]:
#read traffic signs csv
incidents_df = pd.read_csv("..\..\CSV_files\Traffic_Incidents.csv")

#parse for 2018 data
incidents_df = incidents_df[incidents_df.START_DT.str.contains("2018")]
incidents_df

In [ ]:
#parse data for lats and longs
list_long = [i for i in incidents_df["Longitude"]]
list_lat = [i for i in incidents_df["Latitude"]]

In [ ]:
#create map
# map_osm = folium.Map(location = [50.913577283979,-114.073657541927],control_scale = True, zoom_start=10)

#add heatmap
create_heatmap(list_lat, list_long, map_osm, heat_name = "incidents (rainbow)", r = 20)
create_heatmap_incidents(list_lat, list_long, map_osm, heat_name = "incidents", r = 20)
# create_grid_array(grid_array, map_osm)
# create_markers(map_osm, grid_array)
# folium.LayerControl().add_to(map_osm)

# map_osm.save("traffic_heat_map_grid_marker.html")
# map_osm

# Traffic signs (stop, warning yield) specific heat map

In [ ]:
#read traffic signs csv
signs_df = pd.read_csv("..\..\CSV_files\Traffic_Signs.csv")

#drop nan and 0 values in sign count and sign type
signs_df.dropna(subset=["SGN_COUNT_NO"], how='all', inplace=True)
signs_df.dropna(subset=["BLADE_TYPE"], how='all', inplace=True)
signs_df = signs_df[signs_df.SGN_COUNT_NO != 0]

#list of "irrelevant signs" (parking signs, info signs etc.)
# signs_nan = ['Timed Parking','Park Plus','Parking Restrictions','Street Name','Snow Route','Guide / Information','Loading Zone','Disabled Parking','Residential Parking', 'Overhead Guide']

#remove irrelevant signs
signs_df = signs_df.loc[signs_df["BLADE_TYPE"].isin(["Stop","Warning","Yield"])]

signs_df

In [ ]:
#parse data for lats, longs and weight (sign count)
list_long = []
list_lat = []
list_weight = [i for i in signs_df["SGN_COUNT_NO"]]

#parse POINT string
for row in signs_df["POINT"]:
    string_stripped = row.replace("POINT", "").replace("(", "").replace(")", "")
    string_split = string_stripped.split()

    list_long.append(float(string_split[0]))
    list_lat.append(float(string_split[1]))

In [ ]:
#create map
# map_osm = folium.Map(location = [50.913577283979,-114.073657541927],control_scale = True, zoom_start=10)

#add heatmap
create_heatmap(list_lat, list_long, map_osm, heat_name = "signs (stop, warning, yield)", list_weight = list_weight, r = 15)
# create_grid_array(grid_array, map_osm)
# create_markers(map_osm, grid_array)
# folium.LayerControl().add_to(map_osm)

# map_osm.save("traffic_heat_map_grid_marker.html")
# map_osm

# Traffic signs (speed) specific heat map

In [ ]:
#read traffic signs csv
signs_df = pd.read_csv("..\..\CSV_files\Traffic_Signs.csv")

#drop nan and 0 values in sign count and sign type
signs_df.dropna(subset=["SGN_COUNT_NO"], how='all', inplace=True)
signs_df.dropna(subset=["BLADE_TYPE"], how='all', inplace=True)
signs_df = signs_df[signs_df.SGN_COUNT_NO != 0]

#list of "irrelevant signs" (parking signs, info signs etc.)
# signs_nan = ['Timed Parking','Park Plus','Parking Restrictions','Street Name','Snow Route','Guide / Information','Loading Zone','Disabled Parking','Residential Parking', 'Overhead Guide']

#remove irrelevant signs
signs_df = signs_df.loc[signs_df["BLADE_TYPE"].isin(["Speed"])]

signs_df

In [ ]:
#parse data for lats, longs and weight (sign count)
list_long = []
list_lat = []
list_weight = [i for i in signs_df["SGN_COUNT_NO"]]

#parse POINT string
for row in signs_df["POINT"]:
    string_stripped = row.replace("POINT", "").replace("(", "").replace(")", "")
    string_split = string_stripped.split()

    list_long.append(float(string_split[0]))
    list_lat.append(float(string_split[1]))

In [ ]:
#create map
# map_osm = folium.Map(location = [50.913577283979,-114.073657541927],control_scale = True, zoom_start=10)

#add heatmap
create_heatmap(list_lat, list_long, map_osm, heat_name = "signs (speed)", list_weight = list_weight, r = 20)
# create_grid_array(grid_array, map_osm)
# create_markers(map_osm, grid_array)
# folium.LayerControl().add_to(map_osm)

# map_osm.save("traffic_heat_map_grid_marker.html")
# map_osm

# Traffic signs (pedestrians and bicycle pathways) specific heat map

In [ ]:
#read traffic signs csv
signs_df = pd.read_csv("..\..\CSV_files\Traffic_Signs.csv")

#drop nan and 0 values in sign count and sign type
signs_df.dropna(subset=["SGN_COUNT_NO"], how='all', inplace=True)
signs_df.dropna(subset=["BLADE_TYPE"], how='all', inplace=True)
signs_df = signs_df[signs_df.SGN_COUNT_NO != 0]

#list of "irrelevant signs" (parking signs, info signs etc.)
# signs_nan = ['Timed Parking','Park Plus','Parking Restrictions','Street Name','Snow Route','Guide / Information','Loading Zone','Disabled Parking','Residential Parking', 'Overhead Guide']

#remove irrelevant signs
signs_df = signs_df.loc[signs_df["BLADE_TYPE"].isin(['Pedestrian','Bicycle / Pathway'])]

signs_df

In [ ]:
#parse data for lats, longs and weight (sign count)
list_long = []
list_lat = []
list_weight = [i for i in signs_df["SGN_COUNT_NO"]]

#parse POINT string
for row in signs_df["POINT"]:
    string_stripped = row.replace("POINT", "").replace("(", "").replace(")", "")
    string_split = string_stripped.split()

    list_long.append(float(string_split[0]))
    list_lat.append(float(string_split[1]))

In [ ]:
#create map
# map_osm = folium.Map(location = [50.913577283979,-114.073657541927],control_scale = True, zoom_start=10)

#add heatmap
create_heatmap(list_lat, list_long, map_osm, heat_name = "signs (pedestrian, bicycle)", list_weight = list_weight, r = 15)
# create_grid_array(grid_array, map_osm)
# create_markers(map_osm, grid_array)
# folium.LayerControl().add_to(map_osm)

# map_osm.save("traffic_heat_map_grid_marker.html")
# map_osm

# Traffic signs (playground and schools) specific heat map

In [ ]:
#read traffic signs csv
signs_df = pd.read_csv("..\..\CSV_files\Traffic_Signs.csv")

#drop nan and 0 values in sign count and sign type
signs_df.dropna(subset=["SGN_COUNT_NO"], how='all', inplace=True)
signs_df.dropna(subset=["BLADE_TYPE"], how='all', inplace=True)
signs_df = signs_df[signs_df.SGN_COUNT_NO != 0]

#list of "irrelevant signs" (parking signs, info signs etc.)
# signs_nan = ['Timed Parking','Park Plus','Parking Restrictions','Street Name','Snow Route','Guide / Information','Loading Zone','Disabled Parking','Residential Parking', 'Overhead Guide']

#remove irrelevant signs
signs_df = signs_df.loc[signs_df["BLADE_TYPE"].isin(['Playground','School'])]

signs_df

In [ ]:
#parse data for lats, longs and weight (sign count)
list_long = []
list_lat = []
list_weight = [i for i in signs_df["SGN_COUNT_NO"]]

#parse POINT string
for row in signs_df["POINT"]:
    string_stripped = row.replace("POINT", "").replace("(", "").replace(")", "")
    string_split = string_stripped.split()

    list_long.append(float(string_split[0]))
    list_lat.append(float(string_split[1]))

In [ ]:
#create map
# map_osm = folium.Map(location = [50.913577283979,-114.073657541927],control_scale = True, zoom_start=10)

#add heatmap
create_heatmap(list_lat, list_long, map_osm, heat_name = "signs (playground, school)", list_weight = list_weight, r = 15)
# create_grid_array(grid_array, map_osm)
# create_markers(map_osm, grid_array)
# folium.LayerControl().add_to(map_osm)

# map_osm.save("layered_heat_map_grid_marker.html")
# map_osm

# Speed limit road colors

In [ ]:
#import traffic Speed Limits
speed_df = pd.read_csv('..\..\CSV_files\Speed_Limits.csv')

#take relevant columns
speed_df = speed_df[["SPEED","multiline"]]
speed_df

In [ ]:
#speed list
list_speed = [i for i in speed_df["SPEED"]]

In [ ]:
#parse multiline string for coordinate list
list_coord_row =[]

for row in speed_df["multiline"]:

    #strip MULTILINESTRING and ( 
    string = row.replace("MULTILINESTRING","").replace("(","")

    #split on every line
    string_split = string.split("),")

    list_coord = []
    for string in string_split:
        #find coordinate pairs
        pairs = string.split(",")
        list_pair = []

        for pair in pairs:
            #convert each coordinate into a float
            pair_strip = pair.strip().replace(")","")
            pair_split = pair_strip.split()
            float_split = [float(i) for i in pair_split]

            list_pair.append(tuple(float_split))
        list_coord.append([list_pair])
    list_coord_row.append(list_coord)

In [ ]:
#create color gradient
yellow = Color("yellow")
colors = list(yellow.range_to(Color("red"),10))
colors = [str(i) for i in colors]
colors.insert(2,'#ffd400')
colors.insert(4,'#ffb800',)

keys = [i for i in range(20,111,10)]
keys.append(35)
keys.append(45)
keys.sort()

#create dictionary for speeds to corresponding color
colors_dict = dict(zip(keys,colors))

In [ ]:
#creating legend using pyplot
fig, ax = plt.subplots()
fig.set_size_inches(5, 10.5)
handles = [mpatches.Patch(color=colors_dict[x], label=x) for x in colors_dict.keys()]
ax.legend(handles = handles, fontsize = 30, loc = "center", title = "Speed limit (km/h)", title_fontsize = "20")
fig.gca().set_axis_off()
fig.savefig("..\..\HTML_files\legend.png")

In [ ]:
feature_speed = folium.map.FeatureGroup(name = "speed limits", overlay = True, show = True)

#add legend image to html
FloatImage("legend.png", bottom=-10, left=85).add_to(feature_speed)

#add roads and their corresponding color based on the speed limit
for row,speed in zip(list_coord_row, list_speed):
    for line in row:
        for coordinate in line:
            style = {'fillColor': colors_dict[speed], 'color': colors_dict[speed]}
            lines = MultiLineString([coordinate])
            geo = gpd.GeoSeries([lines]).__geo_interface__
            folium.GeoJson(geo, style_function=lambda x, fillColor = style["fillColor"], color = style["color"]: {"fillColor": fillColor, "color": color}).add_to(feature_speed)

In [ ]:
#create map
# map_osm = folium.Map(location = [50.913577283979,-114.073657541927],control_scale = True, zoom_start=10)

#add heatmap
# create_heatmap(list_lat, list_long, map_osm, heat_name = "signs (playground, school)", list_weight = list_weight, r = 15)

#create grid array and markers feature layers
map_osm.add_child(feature_speed)
create_grid_array(grid_array, map_osm)
create_markers(map_osm, grid_array)

#add layer control
folium.LayerControl(position = "bottomleft").add_to(map_osm)

map_osm.save("..\..\HTML_files\layered_heat_map_grid_marker.html")
map_osm